In [1]:
import pandas as pd
import numpy as np

# ===========================
# 1. 基础假设（可修改）
# ===========================
base_year = 2025

base_hw_rev = 307_003
base_sw_rev = 109_158

hw_growth = [0.03, 0.03, 0.03, 0.03, 0.03]
sw_growth = [0.12, 0.11, 0.10, 0.08, 0.06]

m_hw = 0.35
m_sw = 0.62

wacc = 0.10
g_terminal = 0.025

net_debt = -20043
shares_out = 14_848

# ===========================
# 2. 预测收入与 FCFF
# ===========================
years = [base_year]
hw_rev = [base_hw_rev]
sw_rev = [base_sw_rev]
total_rev = [base_hw_rev + base_sw_rev]

hw_yoy = [np.nan]
sw_yoy = [np.nan]

fcff_hw = [base_hw_rev * m_hw]
fcff_sw = [base_sw_rev * m_sw]
fcff_total = [fcff_hw[-1] + fcff_sw[-1]]

for i in range(5):
    year = base_year + i + 1

    next_hw = hw_rev[-1] * (1 + hw_growth[i])
    next_sw = sw_rev[-1] * (1 + sw_growth[i])

    years.append(year)
    hw_rev.append(next_hw)
    sw_rev.append(next_sw)
    total_rev.append(next_hw + next_sw)

    hw_yoy.append(hw_growth[i] * 100)
    sw_yoy.append(sw_growth[i] * 100)

    fc_hw = next_hw * m_hw
    fc_sw = next_sw * m_sw
    fcff_hw.append(fc_hw)
    fcff_sw.append(fc_sw)
    fcff_total.append(fc_hw + fc_sw)

# ===========================
# 3. 折现与终值
# ===========================
discount_factors = [np.nan]
pv_fcff = [np.nan]

for t in range(1, 6):
    df_t = 1 / ((1 + wacc) ** t)
    discount_factors.append(df_t)
    pv_fcff.append(fcff_total[t] * df_t)

fcff_last = fcff_total[-1]
terminal_fcff = fcff_last * (1 + g_terminal)
terminal_value = terminal_fcff / (wacc - g_terminal)
pv_terminal = terminal_value * discount_factors[-1]

# ===========================
# 4. 输出表格
# ===========================
df = pd.DataFrame({
    "Year": years,
    "HW Revenue (B)": [round(x/1000, 2) for x in hw_rev],
    "SW Revenue (B)": [round(x/1000, 2) for x in sw_rev],
    "Total Revenue (B)": [round(x/1000, 2) for x in total_rev],
    "HW YoY (%)": hw_yoy,
    "SW YoY (%)": sw_yoy,
    "FCFF HW (B)": [round(x/1000, 2) for x in fcff_hw],
    "FCFF SW (B)": [round(x/1000, 2) for x in fcff_sw],
    "FCFF Total (B)": [round(x/1000, 2) for x in fcff_total],
    "Discount Factor": discount_factors,
    "PV FCFF (B)": [None if isinstance(x, float)==False else round(x/1000, 2) for x in pv_fcff],
})

display(df)

# ===========================
# 5. 估值
# ===========================
enterprise_value = np.nansum(pv_fcff) + pv_terminal
equity_value = enterprise_value - net_debt

value_per_share = (equity_value * 1_000) / (shares_out * 1_000)

print(f"Enterprise Value: {enterprise_value/1000:.2f} B USD")
print(f"Equity Value:     {equity_value/1000:.2f} B USD")
print(f"DCF Price:        {value_per_share:.2f} USD/share")


,Year,HW Revenue (B),SW Revenue (B),Total Revenue (B),HW YoY (%),SW YoY (%),FCFF HW (B),FCFF SW (B),FCFF Total (B),Discount Factor,PV FCFF (B)
0,2025,307.00,109.16,416.16,NaN,NaN,107.45,67.68,175.13,NaN,NaN
1,2026,316.21,122.26,438.47,3.0,12.0,110.67,75.80,186.47,0.909091,169.52
2,2027,325.70,135.71,461.40,3.0,11.0,113.99,84.14,198.13,0.826446,163.75
3,2028,335.47,149.28,484.75,3.0,10.0,117.41,92.55,209.97,0.751315,157.75
4,2029,345.53,161.22,506.75,3.0,8.0,120.94,99.96,220.89,0.683013,150.87
5,2030,355.90,170.89,526.79,3.0,6.0,124.57,105.95,230.52,0.620921,143.13


Enterprise Value: 2741.18 B USD
Equity Value:     2761.22 B USD
DCF Price:        185.97 USD/share
